<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/sommamtx_colab_widgets_arcdist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# SommaMTX — Widgets con **Restore che svuota** + **Distanza angolare**

Questa versione include:
1. **Ruota un punto** attorno a un polo (box + Run/Restore/Set example).
2. **SommaMTX** (composizione di due poli) (box + Run/Restore/Set example).
3. **Distanza angolare** tra due punti (lat, lon) in gradi, implementata con la **legge dei coseni sferici** (clamping numerico).


In [ ]:

# === Funzioni core ===
import math

def deg2rad(d): return d * math.pi / 180.0
def rad2deg(r): return r * 180.0 / math.pi
def norm_lon_360(lon): return lon % 360.0

def pos_vec(lat, lon):
    th = 90.0 - lat
    z = math.cos(deg2rad(th))
    h = math.cos(deg2rad(lat))
    x = h * math.cos(deg2rad(lon))
    y = h * math.sin(deg2rad(lon))
    return (x, y, z)

def to_pole(v):
    x, y, z = v
    zc = max(-1.0, min(1.0, z))
    lat = rad2deg(math.asin(zc))
    lon = rad2deg(math.atan2(y, x))
    return (lat, norm_lon_360(lon))

def rot_mat(lat, lon, omega):
    ex, ey, ez = pos_vec(lat, lon)
    th = deg2rad(omega)
    c = math.cos(th)
    s = math.sin(th)
    v = 1.0 - c
    return [
        [ex*ex*v + c,     ex*ey*v - ez*s, ex*ez*v + ey*s],
        [ey*ex*v + ez*s,  ey*ey*v + c,    ey*ez*v - ex*s],
        [ez*ex*v - ey*s,  ez*ey*v + ex*s, ez*ez*v + c   ],
    ]

def matmul(A,B):
    return [[sum(A[i][k]*B[k][j] for k in range(3)) for j in range(3)] for i in range(3)]

def matvec(R,v):
    return (R[0][0]*v[0]+R[0][1]*v[1]+R[0][2]*v[2],
            R[1][0]*v[0]+R[1][1]*v[1]+R[1][2]*v[2],
            R[2][0]*v[0]+R[2][1]*v[1]+R[2][2]*v[2])

def matrix_to_pole(R, prefer_positive_angle=True):
    tr = R[0][0] + R[1][1] + R[2][2]
    c = (tr - 1.0)/2.0
    c = max(-1.0, min(1.0, c))
    th = math.acos(c)
    if abs(th) < 1e-15:
        return (0.0,0.0,0.0)
    rx = R[2][1] - R[1][2]
    ry = R[0][2] - R[2][0]
    rz = R[1][0] - R[0][1]
    den = 2.0*math.sin(th)
    kx, ky, kz = rx/den, ry/den, rz/den
    n = math.sqrt(kx*kx+ky*ky+kz*kz)
    if n==0:
        kx, ky, kz = 0,0,1
    else:
        kx, ky, kz = kx/n, ky/n, kz/n
    lat = rad2deg(math.asin(max(-1.0,min(1.0,kz))))
    lon = norm_lon_360(rad2deg(math.atan2(ky,kx)))
    omega = rad2deg(th)
    if not prefer_positive_angle and omega>0:
        omega = -omega
        lat = -lat
        lon = norm_lon_360(lon+180)
    return (lat,lon,omega)

def apply_rotation_to_point(p_lat,p_lon,pole_lat,pole_lon,pole_ang):
    R = rot_mat(pole_lat,pole_lon,pole_ang)
    v = pos_vec(p_lat,p_lon)
    v2 = matvec(R,v)
    return to_pole(v2)

def compose_poles(p2,p1,prefer_positive_angle=True):
    R1 = rot_mat(*p1)   # R1 prima
    R2 = rot_mat(*p2)   # poi R2
    T = matmul(R2,R1)   # T = R2 * R1
    return matrix_to_pole(T,prefer_positive_angle)

def arc_distance_deg(lat1, lon1, lat2, lon2):
    """Distanza angolare in gradi (legge dei coseni sferici, con clamping)."""
    phi1, phi2 = deg2rad(lat1), deg2rad(lat2)
    lam1, lam2 = deg2rad(lon1), deg2rad(lon2)
    num = (math.cos(phi1)*math.cos(phi2) +
           math.sin(phi1)*math.sin(phi2)*math.cos(lam1 - lam2))
    num = max(-1.0, min(1.0, num))
    return rad2deg(math.acos(num))

def parse_float(label, text):
    try:
        return float(text.strip())
    except Exception:
        raise ValueError(f"Valore non valido per {label}: '{text}'. Inserire un numero (usa punto decimale).")

print("Funzioni caricate.")


Funzioni caricate.


## 1) Ruota un punto — Box + Run / Restore (svuota)

In [ ]:

import ipywidgets as widgets
from IPython.display import display, Markdown

w_point_lat = widgets.Text(description="Point Lat", placeholder="es. 35.3")
w_point_lon = widgets.Text(description="Point Lon", placeholder="es. 160.5")
w_pole_lat  = widgets.Text(description="Pole Lat",  placeholder="es. 88.4")
w_pole_lon  = widgets.Text(description="Pole Lon",  placeholder="es. -27.7 (o 332.3)")
w_pole_ang  = widgets.Text(description="Pole Ang",  placeholder="es. -38.1")

btn_run_point = widgets.Button(description="Run", button_style='success', icon='play')
btn_restore_point = widgets.Button(description="Restore", button_style='warning', icon='history')
btn_example_point = widgets.Button(description="Set example", icon='check')

out_point = widgets.Output()

def _run_point(_=None):
    with out_point:
        out_point.clear_output()
        try:
            plat = parse_float("Point Lat", w_point_lat.value)
            plon = parse_float("Point Lon", w_point_lon.value)
            polat = parse_float("Pole Lat", w_pole_lat.value)
            polon = parse_float("Pole Lon", w_pole_lon.value)
            poang = parse_float("Pole Ang", w_pole_ang.value)
            lat2, lon2 = apply_rotation_to_point(plat, plon, polat, polon, poang)
            display(Markdown(f"**Output** → lat = `{lat2:.6f}`, lon = `{lon2:.6f}` (0–360°E)"))
        except Exception as e:
            display(Markdown(f"<span style='color:red'>Errore: {e}</span>"))

def _restore_point(_=None):
    w_point_lat.value = ""
    w_point_lon.value = ""
    w_pole_lat.value  = ""
    w_pole_lon.value  = ""
    w_pole_ang.value  = ""
    with out_point:
        out_point.clear_output()
        display(Markdown("*Campi svuotati.*"))

def _example_point(_=None):
    w_point_lat.value = "35.3"
    w_point_lon.value = "160.5"
    w_pole_lat.value  = "88.4"
    w_pole_lon.value  = "-27.7"
    w_pole_ang.value  = "-38.1"
    with out_point:
        out_point.clear_output()
        display(Markdown("*Esempio impostato.*"))

btn_run_point.on_click(_run_point)
btn_restore_point.on_click(_restore_point)
btn_example_point.on_click(_example_point)

display(widgets.VBox([
    widgets.HTML("<b>Inserisci i dati e premi <i>Run</i>. <i>Restore</i> svuota i box.</b>"),
    widgets.HBox([w_point_lat, w_point_lon]),
    widgets.HBox([w_pole_lat,  w_pole_lon, w_pole_ang]),
    widgets.HBox([btn_run_point, btn_restore_point, btn_example_point]),
    out_point
]))


## 2) SommaMTX — Composizione poli (T = R₂ × R₁) — Box + Run / Restore (svuota)

In [ ]:

import ipywidgets as widgets
from IPython.display import display, Markdown

w_p1_lat = widgets.Text(description="P1 Lat", placeholder="es. 0.0")
w_p1_lon = widgets.Text(description="P1 Lon", placeholder="es. 211.7")
w_p1_ang = widgets.Text(description="P1 Ang", placeholder="es. -54.9")

w_p2_lat = widgets.Text(description="P2 Lat", placeholder="es. 6.8")
w_p2_lon = widgets.Text(description="P2 Lon", placeholder="es. 214.3")
w_p2_ang = widgets.Text(description="P2 Ang", placeholder="es. -55.4")

btn_run_comp = widgets.Button(description="Run", button_style='success', icon='play')
btn_restore_comp = widgets.Button(description="Restore", button_style='warning', icon='history')
btn_example_comp = widgets.Button(description="Set example", icon='check')

out_comp = widgets.Output()

def _run_comp(_=None):
    with out_comp:
        out_comp.clear_output()
        try:
            p1 = (parse_float("P1 Lat", w_p1_lat.value),
                  parse_float("P1 Lon", w_p1_lon.value),
                  parse_float("P1 Ang", w_p1_ang.value))
            p2 = (parse_float("P2 Lat", w_p2_lat.value),
                  parse_float("P2 Lon", w_p2_lon.value),
                  parse_float("P2 Ang", w_p2_ang.value))
            lat, lon, ang = compose_poles(p2, p1)
            display(Markdown(f"**Polo risultante** → lat = `{lat:.6f}`, lon = `{lon:.6f}`, ang = `{ang:.6f}`"))
        except Exception as e:
            display(Markdown(f"<span style='color:red'>Errore: {e}</span>"))

def _restore_comp(_=None):
    w_p1_lat.value = ""
    w_p1_lon.value = ""
    w_p1_ang.value = ""
    w_p2_lat.value = ""
    w_p2_lon.value = ""
    w_p2_ang.value = ""
    with out_comp:
        out_comp.clear_output()
        display(Markdown("*Campi svuotati.*"))

def _example_comp(_=None):
    w_p1_lat.value = "0.0"
    w_p1_lon.value = "211.7"
    w_p1_ang.value = "-54.9"
    w_p2_lat.value = "6.8"
    w_p2_lon.value = "214.3"
    w_p2_ang.value = "-55.4"
    with out_comp:
        out_comp.clear_output()
        display(Markdown("*Esempio impostato.*"))

btn_run_comp.on_click(_run_comp)
btn_restore_comp.on_click(_restore_comp)
btn_example_comp.on_click(_example_comp)

display(widgets.VBox([
    widgets.HTML("<b>Inserisci i due poli e premi <i>Run</i>. <i>Restore</i> svuota i box.</b>"),
    widgets.HBox([w_p1_lat, w_p1_lon, w_p1_ang]),
    widgets.HBox([w_p2_lat, w_p2_lon, w_p2_ang]),
    widgets.HBox([btn_run_comp, btn_restore_comp, btn_example_comp]),
    out_comp
]))


## 3) Distanza angolare tra due punti (lat, lon) — Box + Run / Restore (svuota)

In [ ]:

import ipywidgets as widgets
from IPython.display import display, Markdown

w_a_lat = widgets.Text(description="Lat A", placeholder="es. 45.0")
w_a_lon = widgets.Text(description="Lon A", placeholder="es. 9.0 (Milano ≈ 9E)")
w_b_lat = widgets.Text(description="Lat B", placeholder="es. 40.0")
w_b_lon = widgets.Text(description="Lon B", placeholder="es. -74.0 (NY ≈ 74W)")

btn_run_dist = widgets.Button(description="Run", button_style='success', icon='play')
btn_restore_dist = widgets.Button(description="Restore", button_style='warning', icon='history')
btn_example_dist = widgets.Button(description="Set example", icon='check')

out_dist = widgets.Output()

def _run_dist(_=None):
    with out_dist:
        out_dist.clear_output()
        try:
            la1 = parse_float("Lat A", w_a_lat.value)
            lo1 = parse_float("Lon A", w_a_lon.value)
            la2 = parse_float("Lat B", w_b_lat.value)
            lo2 = parse_float("Lon B", w_b_lon.value)
            d = arc_distance_deg(la1, lo1, la2, lo2)
            display(Markdown(f"**Distanza angolare** = `{d:.6f}°`"))
        except Exception as e:
            display(Markdown(f"<span style='color:red'>Errore: {e}</span>"))

def _restore_dist(_=None):
    w_a_lat.value = ""
    w_a_lon.value = ""
    w_b_lat.value = ""
    w_b_lon.value = ""
    with out_dist:
        out_dist.clear_output()
        display(Markdown("*Campi svuotati.*"))

def _example_dist(_=None):
    # Milano (45.4642N, 9.19E) vs New York (40.7128N, 74.006W ≈ -74.006)
    w_a_lat.value = "45.4642"
    w_a_lon.value = "9.19"
    w_b_lat.value = "40.7128"
    w_b_lon.value = "-74.006"
    with out_dist:
        out_dist.clear_output()
        display(Markdown("*Esempio impostato (Milano–New York).*"))

btn_run_dist.on_click(_run_dist)
btn_restore_dist.on_click(_restore_dist)
btn_example_dist.on_click(_example_dist)

display(widgets.VBox([
    widgets.HTML("<b>Inserisci le due coordinate e premi <i>Run</i>. <i>Restore</i> svuota i box.</b>"),
    widgets.HBox([w_a_lat, w_a_lon]),
    widgets.HBox([w_b_lat, w_b_lon]),
    widgets.HBox([btn_run_dist, btn_restore_dist, btn_example_dist]),
    out_dist
]))
